In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-26 19:12:05--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-26 19:12:05 (12.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [13]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)  

In [ ]:
SELECT * WHERE course ='data-engineering-zoomcamp';

In [17]:
q ='the course has already started,can I still enroll ?'

In [14]:
index.fit(documents)

In [22]:
boost = {'question':3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)    

In [23]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [28]:
import os

# Set the environment variable
os.environ['OPENAI_API_KEY'] = 'sk-proj-iXuGQ79QRlRkIRv0qmQST3BlbkFJOudQ6KbXmPhfSySOVSPO'

In [29]:
from openai import OpenAI

In [30]:
client = OpenAI()

In [31]:
q

'the course has already started,can I still enroll ?'

In [32]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content": q}]
)    

In [43]:
response.choices[0].message.content

"The ability to enroll in a course that has already started can depend on several factors, such as the policies of the institution offering the course, the nature of the course content, and the instructor's discretion.\n\nHere are some steps you can take to find out if you can still enroll:\n\n1. **Check the institution's policies**: Look up the registration and enrollment guidelines on the institution's website. Some institutions have a grace period during which you can still enroll in a course after it has started.\n\n2. **Contact the course instructor**: Instructors sometimes have the flexibility to allow late enrollments based on their assessment of whether you can catch up on the missed material.\n\n3. **Speak with an academic advisor**: Advisors can provide guidance and might be able to facilitate your late enrollment if it’s allowed within the institution's policies.\n\n4. **Assess the course structure**: If the course is self-paced or has a lot of online materials, you might fi

In [50]:
prompt_template = """
You're a course teaching assistant.Answer the QUESTION based on the CONTEXT.Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesnot contain the answer,output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()


In [45]:
context =""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    

In [52]:
prompt = prompt_template.format(question=q, context=context).strip()

In [53]:
print(prompt)

You're a course teaching assistant.Answer the QUESTION based on the CONTEXT.Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesnot contain the answer,output NONE

QUESTION: the course has already started,can I still enroll ?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

sec

In [54]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content": prompt}]
)

response.choices[0].message.content

"Yes, even if the course has already started, you can still enroll and are eligible to submit the homework. Be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."